In [1]:
project_id = 'elife-data-pipeline'
source_dataset = 'de_dev'
output_dataset = 'de_dev'
output_table_prefix = 'data_science_'
manuscript_min_tf = 10
manuscript_max_tf = 0.9
state_path = 's3://ci-elife-data-pipeline/airflow-config/keyword-extraction/state-dev'

In [2]:
import os
from functools import partial
from itertools import groupby
from typing import List, Tuple, TypeVar

import joblib
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

import data_science_pipeline.configure_warnings

from data_science_pipeline.sql import get_sql
from data_science_pipeline.utils.bq import to_gbq
from data_science_pipeline.utils.io import load_object_from
from data_science_pipeline.utils.misc import identity_fn
from data_science_pipeline.utils.jupyter import (
    read_big_query as _read_big_query
)
from data_science_pipeline.peerscout.models import (
    WeightedKeywordModel
)

In [3]:
model_path = os.path.join(state_path, 'senior_editor_model.joblib')
recommendation_output_table_name = '{output_dataset}.{prefix}{suffix}'.format(
    output_dataset=output_dataset,
    prefix=output_table_prefix,
    suffix='editor_recommendation'
)

In [4]:
print('loading model from:', model_path)
model_dict = load_object_from(model_path)
model_dict.keys()

loading model from: s3://ci-elife-data-pipeline/airflow-config/keyword-extraction/state-dev/senior_editor_model.joblib


dict_keys(['editor_tf_idf_vectorizer', 'editor_tf_idf', 'editor_names', 'editor_person_ids'])

In [5]:
editor_tf_idf_vectorizer = model_dict['editor_tf_idf_vectorizer']
editor_tf_idf_vectorizer

TfidfVectorizer(lowercase=False, smooth_idf=False, token_pattern=None,
                tokenizer=<function identity_fn at 0x7f6921114dd0>)

In [6]:
editor_tf_idf = model_dict['editor_tf_idf']
editor_tf_idf

<64x19284 sparse matrix of type '<class 'numpy.float64'>'
	with 304879 stored elements in Compressed Sparse Row format>

In [7]:
editor_names = model_dict['editor_names']
editor_names

0           Aleksandra Walczak
1                  Andrew King
2               Anna Akhmanova
3     Barbara Shinn-Cunningham
4                Carla Rothlin
                ...           
59                 Tamar Makin
60             Timothy Behrens
61              Utpal Banerjee
62              Vivek Malhotra
63               Wendy Garrett
Name: name, Length: 64, dtype: object

In [8]:
editor_person_ids = model_dict['editor_person_ids']
editor_person_ids

0     50904
1     14601
2      8518
3     19576
4     44396
      ...  
59    18331
60     1044
61     1042
62     1133
63    28627
Name: person_id, Length: 64, dtype: object

In [9]:
editor_person_id_by_name_map = dict(zip(editor_names, editor_person_ids))
editor_person_id_by_name_map

{'Aleksandra Walczak': '50904',
 'Andrew King': '14601',
 'Anna Akhmanova': '8518',
 'Barbara Shinn-Cunningham': '19576',
 'Carla Rothlin': '44396',
 'Catherine Dulac': '1014',
 'Chris Baker': '28129',
 'Christian Büchel': '16197',
 'Christian Hardtke': '1102',
 'Christian Rutz': '15332',
 'Clifford Rosen': '48282',
 'Cynthia Wolberger': '5005',
 'David Ron': '1174',
 'Detlef Weigel': '1030',
 'Didier Stainier': '7189',
 'Diethard Tautz': '1191',
 'Dominique Soldati-Favre': '41647',
 'Eduardo Franco': '1086',
 'Edward Morrisey': '55645',
 'Floris de Lange': '28130',
 'Gary Westbrook': '1202',
 'George Perry': '42011',
 'Gisela Storz': '1188',
 'Huda Zoghbi': '1029',
 'James Manley': '1020',
 'Jessica Tyler': '1421',
 'John Huguenard': '13947',
 'John Kuriyan': '1018',
 'Jonathan Cooper': '1062',
 'Jos van der Meer': '41027',
 'Joshua Gold': '17965',
 'José Faraldo-Gómez': '13987',
 'K VijayRaghavan': '1027',
 'Karla Kirkegaard': '3645',
 'Kate Wassum': '33127',
 'Kathryn Cheah': '91149

In [10]:
weighted_keyword_valid_model = WeightedKeywordModel.from_tf_matrix(
    editor_tf_idf.todense(),
    vectorizer=editor_tf_idf_vectorizer,
    choices=editor_names
)
weighted_keyword_valid_model

In [11]:
read_big_query = partial(_read_big_query, project_id=project_id)

In [12]:
default_query_props = dict(project=project_id, dataset=source_dataset)

In [13]:
manuscript_version_for_recommendation_df = read_big_query(
    get_sql('manuscript-version-initial-submissions-for-senior-editor-recommendation.sql').format(
        **default_query_props
    )
)
manuscript_version_for_recommendation_df.head()

> ```sql
> WITH t_manuscript_version_abstract_keywords AS (
>   SELECT
>     LPAD(CAST(manuscript_abstract_keywords.manuscript_id AS STRING), 5, '0') AS manuscript_id,
>     manuscript_abstract_keywords.version_id,
>     manuscript_abstract_keywords.extracted_keywords,
>     ROW_NUMBER() OVER (
>       PARTITION BY version_id
>       ORDER BY data_hub_imported_timestamp DESC
>     ) AS version_id_row_number
>   FROM `elife-data-pipeline.de_dev.manuscript_abstract_keywords` AS manuscript_abstract_keywords
>   WHERE ARRAY_LENGTH(extracted_keywords) > 0
> ),
> 
> t_last_manuscript_version_abstract_keywords AS (
>   SELECT
>     * EXCEPT(version_id_row_number)
>   FROM t_manuscript_version_abstract_keywords
>   WHERE version_id_row_number = 1
>   ORDER BY version_id
> )
> 
> SELECT version.version_id, manuscript_version_abstract_keywords.extracted_keywords
> FROM `elife-data-pipeline.de_dev.mv_manuscript_version` AS version
> JOIN t_last_manuscript_version_abstract_keywords AS manuscript_version_abstract_keywords
>   ON manuscript_version_abstract_keywords.version_id = version.version_id
> WHERE version.overall_stage = 'Initial Submission'
>   AND ARRAY_LENGTH(version.senior_editors) = 0
>   AND TIMESTAMP_DIFF(CURRENT_TIMESTAMP, version.created_timestamp, DAY) < 365
> ```

Downloading: 100%|██████████| 135/135 [00:01<00:00, 118.95rows/s]


,version_id,extracted_keywords
0,49069/2019-06-05T12:39:51Z,"[1 expression level, accurate, accurate model,..."
1,49224/2019-06-11T08:05:08Z,"[activity, association, bacteria, charge, char..."
2,49391/2019-06-17T03:54:44Z,"[4.1 protein, associate, associated protein, b..."
3,50111/2019-07-11T05:25:04Z,"[activity, axis, basal, basal expression, basa..."
4,51945/2019-09-17T10:10:25Z,"[activation, activation signaling, adhesion, a..."


In [14]:
keyword_similarity = cosine_similarity(
    editor_tf_idf_vectorizer.transform(
        manuscript_version_for_recommendation_df
        ['extracted_keywords']
    ),
    editor_tf_idf
)
print(keyword_similarity.max())
keyword_similarity

0.31383530383438885


array([[0.17501392, 0.09257564, 0.12925561, ..., 0.11115715, 0.11223832,
        0.12264317],
       [0.07085491, 0.05937612, 0.10026875, ..., 0.07263001, 0.10828115,
        0.11347466],
       [0.07943454, 0.05150589, 0.16412044, ..., 0.09477105, 0.18524423,
        0.11168855],
       ...,
       [0.09638545, 0.05975314, 0.08223231, ..., 0.06478368, 0.0645778 ,
        0.13420128],
       [0.03275795, 0.0167641 , 0.00808092, ..., 0.00872577, 0.00869329,
        0.00940108],
       [0.08489603, 0.11403938, 0.07425474, ..., 0.05514426, 0.05960043,
        0.07200308]])

In [15]:
weighted_keyword_valid_model.predict_ranking(
    manuscript_version_for_recommendation_df['extracted_keywords'][:1],
).proba_matrix

matrix([[0.17501392, 0.09257564, 0.12925561, 0.07772865, 0.08019251,
         0.10943372, 0.05140947, 0.09145685, 0.10058674, 0.06017248,
         0.10793771, 0.09086184, 0.13090995, 0.1335926 , 0.12795519,
         0.1083802 , 0.09006374, 0.12840778, 0.12467069, 0.08129095,
         0.10473093, 0.03951719, 0.11307342, 0.12108902, 0.11053364,
         0.11152655, 0.08207643, 0.09327424, 0.13313865, 0.09631412,
         0.0899288 , 0.08612289, 0.11137705, 0.08206186, 0.08064677,
         0.1142563 , 0.07572255, 0.12816551, 0.08741873, 0.11936011,
         0.11055727, 0.137961  , 0.1091965 , 0.09044308, 0.11007885,
         0.19899241, 0.09807388, 0.07881459, 0.12575876, 0.13465324,
         0.06487969, 0.1253848 , 0.09900872, 0.08563235, 0.13102614,
         0.10295128, 0.12309406, 0.10698688, 0.12687547, 0.05387842,
         0.09415986, 0.11115715, 0.11223832, 0.12264317]])

In [16]:
manuscript_matching_keywords_list = weighted_keyword_valid_model.predict_ranking(
    manuscript_version_for_recommendation_df['extracted_keywords']
).matching_keywords_list
pd.Series(manuscript_matching_keywords_list[:5])

0    [[(0.01971169783449064, cell), (0.018013482072...
1    [[(0.00990378673220786, mechanism), (0.0086699...
2    [[(0.02606022951388943, cell), (0.012629204148...
3    [[(0.02272844153225087, cell), (0.011014566388...
4    [[(0.01827395584749503, model), (0.00969073416...
dtype: object

In [17]:
T = TypeVar('T')


def get_recommended_editors_with_probability(
        proba_matrix: List[List[float]],
        manuscript_matching_keywords_list: List[List[Tuple[float, str]]],
        indices: List[T],
        threshold: float = 0.5) -> List[List[Tuple[float, T]]]:
    return [
        sorted([
            (
                p,
                key,
                sum(
                    s for s, _ in editor_matching_keywords
                ),
                editor_matching_keywords
            )
            for p, key, editor_matching_keywords in zip(
                row,
                indices,
                editors_matching_keywords
            ) if p >= threshold
        ], reverse=True)
        for row, editors_matching_keywords in zip(proba_matrix, manuscript_matching_keywords_list)
    ]


prediction_results_with_similarity = pd.Series(
    get_recommended_editors_with_probability(
        keyword_similarity,
        manuscript_matching_keywords_list,
        editor_names,
        threshold=0.001
    ),
    index=manuscript_version_for_recommendation_df.index
)
# print(prediction_results_with_similarity[0])
prediction_results_with_similarity[:5]

0    [(0.1989924120958892, Naama Barkai, 0.19899241...
1    [(0.15570505099555257, Philip Cole, 0.15570505...
2    [(0.25605465592054394, Dominique Soldati-Favre...
3    [(0.26097569460407843, Maureen Murphy, 0.26097...
4    [(0.22019632285697605, Richard Aldrich, 0.2201...
dtype: object

In [18]:
prediction_results_df = pd.concat([
    manuscript_version_for_recommendation_df['version_id'],
    prediction_results_with_similarity.to_frame('prediction'),
], axis=1)
print(len(prediction_results_df))
prediction_results_df.head()

135


,version_id,prediction
0,49069/2019-06-05T12:39:51Z,"[(0.1989924120958892, Naama Barkai, 0.19899241..."
1,49224/2019-06-11T08:05:08Z,"[(0.15570505099555257, Philip Cole, 0.15570505..."
2,49391/2019-06-17T03:54:44Z,"[(0.25605465592054394, Dominique Soldati-Favre..."
3,50111/2019-07-11T05:25:04Z,"[(0.26097569460407843, Maureen Murphy, 0.26097..."
4,51945/2019-09-17T10:10:25Z,"[(0.22019632285697605, Richard Aldrich, 0.2201..."


In [19]:
prediction_results_df['prediction'][0]

[(0.1989924120958892,
  'Naama Barkai',
  0.19899241209588905,
  [(0.0226742815801745, 'cell'),
   (0.016482479414031415, 'model'),
   (0.009703823896600103, 'dynamic'),
   (0.009511668967954556, 'expression'),
   (0.00903128164634069, 'network'),
   (0.007635950914975006, 'level'),
   (0.006471934007204426, 'role'),
   (0.006332251978271956, 'response'),
   (0.006239130625650309, 'factor'),
   (0.006097704429136512, 'molecular'),
   (0.005531555455601166, 'mathematical'),
   (0.00539808599353599, 'cancer'),
   (0.004736055416506068, 'prediction'),
   (0.004563679555331732, 'type'),
   (0.003693516833203756, 'stress'),
   (0.003594251394467917, 'concentration'),
   (0.00339584212934561, 'heterogeneity'),
   (0.0032666337869742923, 'disease'),
   (0.0031131070451288797, 'variation'),
   (0.003048852214568256, 'measurement'),
   (0.0028839296667226163, 'phenotypic heterogeneity'),
   (0.0028743055290256572, 'phenotypic'),
   (0.002641146372309842, 'mathematical modeling'),
   (0.00261627

In [20]:
prediction_results_flat_df = pd.DataFrame([
    {
        'version_id': row.version_id,
        'score': predicted_editor[0],
        'name': predicted_editor[1],
        'person_id': editor_person_id_by_name_map[predicted_editor[1]],
        'matching_keyword_score': predicted_editor[2],
        'matching_keywords': [{
            'score': keyword_score,
            'keyword': keyword
        } for keyword_score, keyword in predicted_editor[3]],
    }
    for row in prediction_results_df.itertuples()
    for predicted_editor in row.prediction
])
print(len(prediction_results_flat_df))
prediction_results_flat_df.head()

8503


,version_id,score,name,person_id,matching_keyword_score,matching_keywords
0,49069/2019-06-05T12:39:51Z,0.198992,Naama Barkai,1725,0.198992,"[{'score': 0.0226742815801745, 'keyword': 'cel..."
1,49069/2019-06-05T12:39:51Z,0.175014,Aleksandra Walczak,50904,0.175014,"[{'score': 0.01971169783449064, 'keyword': 'ce..."
2,49069/2019-06-05T12:39:51Z,0.137961,Maureen Murphy,33764,0.137961,"[{'score': 0.026324886179440718, 'keyword': 'c..."
3,49069/2019-06-05T12:39:51Z,0.134653,Philip Cole,21020,0.134653,"[{'score': 0.018939014804428538, 'keyword': 'c..."
4,49069/2019-06-05T12:39:51Z,0.133593,Detlef Weigel,1030,0.133593,"[{'score': 0.011260566558338708, 'keyword': 'e..."


In [21]:
prediction_results_flat_df['version_id'].nunique()

134

In [22]:
prediction_results_flat_df.max()

version_id                58638/2020-05-06T09:35:13Z
score                                       0.313835
name                                   Wendy Garrett
person_id                                      97155
matching_keyword_score                      0.313835
dtype: object

In [23]:
print('writing to:', recommendation_output_table_name)
to_gbq(
    prediction_results_flat_df,
    recommendation_output_table_name,
    project_id=project_id,
    if_exists='replace'
)
print('done')

writing to: de_dev.data_science_editor_recommendation
done
